In [2]:
# Data preparation
# Import modules
import sys
# If your authentification script is not in the project directory
# append its folder to sys.path
sys.path.append("../spotify_api_web_app")
import authorization
import pandas as pd
from tqdm import tqdm
import time

# Authorize and call access object "sp"
sp = authorization.authorize()

# Get all genres
genres = sp.recommendation_genre_seeds()

# Set number of recommendations per genre
n_recs = 50

# Initiate a dictionary with all the information you want to crawl
data_dict = {"id":[], "genre":[], "track_name":[], "artist_name":[],
             "valence":[],"acousticness":[],"danceability":[],
             "instrumentalness":[],"key":[],"liveness":[],"loudness":[],
             "speechiness":[],"tempo":[], "energy":[]}

## CRAWL DATA ##

# Get recs for every genre
for g in tqdm(genres):
    
    # Get n recommendations
    recs = sp.recommendations(genres = [g], limit = n_recs)
    
    # json-like string to dict
    recs = eval(recs.json().replace("null", "-999").replace("false", "False").replace("true", "True"))["tracks"]

    # Crawl data from each track
    for track in recs:
        
        # ID and Genre
        data_dict["id"].append(track["id"])
        data_dict["genre"].append(g)
        
        # Metadata
        track_meta = sp.track(track["id"])
        data_dict["track_name"].append(track_meta.name)
        data_dict["artist_name"].append(track_meta.album.artists[0].name)
        
        # Audio Features
        track_features = sp.track_audio_features(track["id"])
        data_dict["valence"].append(track_features.valence)
        data_dict["acousticness"].append(track_features.acousticness)
        data_dict["danceability"].append(track_features.danceability)
        data_dict["instrumentalness"].append(track_features.instrumentalness)
        data_dict["key"].append(track_features.key)
        data_dict["liveness"].append(track_features.liveness)
        data_dict["loudness"].append(track_features.loudness)
        data_dict["speechiness"].append(track_features.speechiness)
        data_dict["tempo"].append(track_features.tempo)
        data_dict["energy"].append(track_features.energy)

        

# ## PROCESS DATA ##

# # Store data in dataframe


# # Drop duplicates
# df.drop_duplicates(subset = "id", keep = "first", inplace = True)
# df.to_csv("valence_arousal_dataset.csv", index = False)

100%|████████████████████████████████████████| 126/126 [30:15<00:00, 14.41s/it]


In [5]:
df = pd.DataFrame(data_dict)

In [7]:
df.drop_duplicates(subset = "id", keep = "first", inplace = True)

In [8]:
df.to_csv("dataset_af.csv", index = False)

In [10]:
fd = pd.read_csv('dataset_af.csv')

In [14]:
fd.head()

,id,genre,track_name,artist_name,valence,acousticness,danceability,instrumentalness,key,liveness,loudness,speechiness,tempo,energy
0,4JKHACLkotsxdKNF9vJYOS,acoustic,Nothing Arrived - Live from Spotify London,Villagers,0.420,0.929000,0.579,0.00180,7,0.6710,-11.599,0.0397,102.713,0.415
1,7utRJ4BeYx85khzP3lKoBX,acoustic,Tears in Heaven,Eric Clapton,0.312,0.835000,0.498,0.53600,9,0.1330,-12.238,0.0257,153.744,0.242
2,4vdnC0qoyb89SmCow0OT4m,acoustic,Bad Romance,Lissie,0.275,0.000565,0.409,0.00075,9,0.6900,-5.109,0.0290,116.207,0.631
3,2ZheGCM31EbCwUfGs0WJB1,acoustic,Barton Hollow,The Civil Wars,0.435,0.123000,0.403,0.00000,2,0.0857,-7.338,0.0303,90.854,0.659
4,4Pl8ViAolRF4rK8QBMhKd9,acoustic,Redemption Song,Bob Marley & The Wailers,0.698,0.585000,0.666,0.00000,0,0.0790,-10.721,0.0400,116.082,0.259


In [15]:
fd.shape

(5858, 14)

In [20]:
fd.isnull().sum()

id                  0
genre               0
track_name          2
artist_name         2
valence             0
acousticness        0
danceability        0
instrumentalness    0
key                 0
liveness            0
loudness            0
speechiness         0
tempo               0
energy              0
dtype: int64